# OrientDB tutorial

## Prerequisites

### Documentation

You will find all documentation for :
* [OrientDB SQL reference](http://www.orientdb.com/docs/last/SQL-Functions.html)
* [Orientdb python client](http://orientdb.com/docs/last/PyOrient-Client.html#working-with-the-client)

## Import libraries

In [42]:
import pyorient

In [43]:
ROOT_PASSWORD = "root"
client = pyorient.OrientDB("localhost", 2424)
session_id = client.connect("root", ROOT_PASSWORD)

In [8]:
print(client.db_list())

{{'databases': {}}}


## I. Quick start

### Creating the database

**Q:** Create a database `gods` as a `GRAPH_DATABASE` in `MEMORY_STORAGE_TYPE`. 

We will use it to store relationships between Greek deities.

In [9]:
db = client.db_create("gods",type=pyorient.DB_TYPE_GRAPH,storage=pyorient.STORAGE_TYPE_MEMORY)

**Q:** Connect your pyorient client to the `gods` database.

In [10]:
client.db_open("gods","root",ROOT_PASSWORD)

**Q:** You should now be able to launch OrientDB queries through the Python client with the [command()](http://orientdb.com/docs/last/PyOrient-Client-Command.html) function. 

You should think of OrientDB as a Graph-Document database for the following questions. Each vertex and edge will contain information on it inside a JSON document.

Create a new Vertex with content `{name: 'Zeus', symbol: 'thunder'}`. The [CREATE VERTEX : Create a vertex using JSON content](http://orientdb.com/docs/last/SQL-Create-Vertex.html) doc page should help you.

In [11]:
client.command("create vertex set name = 'Zeus', symbol = 'thunder'")

You have created a VERTEX in the previous question. The VERTEX is a [class](https://orientdb.com/docs/last/Tutorial-Classes.html) of OrientDB which defines a record that can be linked to others through EDGE instances.

You can find all VERTEX created in the database with a SQL command on the `V` table, like `SELECT * FROM V`. 

**Q:** Print all current vertices in `gods`, it should only have `Zeus` though for now.

In [12]:
[item.name for item in client.command("SELECT * FROM V")]

['Zeus']

**Q:** Create new vertices with content : 
```
{name:Héra, symbol:tiara}
{name:Poséidon, symbol:trident}
{name:Athena, symbol:helmet}
{name:Arès, symbol:weapons} 
```

In [13]:
client.command("create vertex set name = 'Hera', symbol = 'tiara'")
client.command("create vertex set name = 'Poséidon', symbol = 'trident'")
client.command("create vertex set name = 'Athena', symbol = 'helmet'")
client.command("create vertex set name = 'Arès', symbol = 'weapons'")

**Q:** Display all vertices with name = `Arès`

In [14]:
[item for item in client.command("SELECT * FROM V where name='Arès'")]

**Q:** Create an EDGE from `Zeus` to `Poséidon` with the content `{kind: 'sibling'}

In [15]:
client.command("create edge FROM (SELECT FROM V where name='Zeus') TO (SELECT FROM V where name='Poséidon') SET kind='sibling'")

**Q:** Redisplay all vertices, discuss.

In [16]:
[item for item in client.command("SELECT * FROM V")]

**Q:** Display all edges. They are contained in the class `E`

In [17]:
[item for item in client.command('select * from e')]

Two fields on vertices have appeared, containing the outgoing (out_) and incoming (in_) links.

At the edge level, two fields point to the original (out) and destination (in) vertices.

**Q:** Lets create some more edges :

* Zeus > Héra (sibling)
* Zeus > Arès (father)
* Zeus > Athena (father)
* Héra > Arès (mother)
* Héra > Zeus (sibling)
* Poséidon > Zeus (sibling)

_Hint 1 :_ check [the CREATE EDGE doc page](http://orientdb.com/docs/last/SQL-Create-Edge.html) to find an example for creating edges on vertices using subqueries so you can run queries to fetch the vertices before creating an edge in between.

_Hint 2 :_ after you have found the command to create edges between vertices with sub-queries, you should be well-versed enough in Python to create a list of all edges in the question, and loop the command on each element of the list to create all edges in one go =)

In [18]:
def createEdge(_from,_to,_kind):
    return_val = 0
    try:
        client.command(f"create edge FROM (SELECT FROM V where name='{_from}') TO (SELECT FROM V where name='{_to}') SET kind='{_kind}'")
    except Exception as e:
        return_val=-1
    finally:
        return return_val


In [19]:
l_Edges = [
            {"from":'Zeus',"to":'Hera',"kind":'sibling'},
            {"from":'Zeus',"to":'Arès',"kind":'father'},
            {"from":'Zeus',"to":'Athena',"kind":'father'},
            {"from":'Hera',"to":'Arès',"kind":'mother'},
            {"from":'Hera',"to":'Zeus',"kind":'sibling'},   
            {"from":'Poséidon',"to":'Zeus',"kind":'sibling'}    
        ]

In [20]:
for dict in l_Edges:
    createEdge(dict["from"],dict["to"],dict["kind"])

### Looking for data

**Q:** Using [out()](http://orientdb.com/docs/last/Tutorial-Working-with-graphs.html#querying-graphs) function, display all vertices connected and outgoing from Zeus.

You should use the EXPAND() special function to transform the vertex collection in the result-set by expanding it, making the results more readable.

In [23]:
[item.out for item in client.command("SELECT out().name from V where name='Zeus'")][0]

['Poséidon', 'Hera', 'Arès', 'Athena']

**Q:** Display all vertices which got a father (the vertices which are the destination of an arc whose kind attribute is father).

_Hint: You can notice that we use the field `in` the arc, and not the function `in()` which applies to vertices._

In [24]:
[item.name for item in client.command("SELECT EXPAND(in) FROM E WHERE kind='father'")]

['Arès', 'Athena']

**Q:** As in SQL, the operator `in` used in a clause `where` allows to restrict the possible values with an embedded query _(where ... in (select ...))_. 

Display the mothers, by displaying the vertices where an outgoing arc is part of the arcs where kind is a mother.

In [25]:
[item.name for item in client.command("SELECT * from V where @rid in (SELECT out FROM E where kind='mother')")][0]

'Hera'

**Q:** Display the brothers and sisters of Zeus (the destination summits of an arc whose kind is sibling and whose original summit is Zeus).

In [27]:
[item.name for item in client.command("SELECT * from V where @rid in (SELECT out FROM E where kind='sibling') and @rid NOT IN (Select @rid from V where name = 'Zeus')")]

['Hera', 'Poséidon']

## Modeling a Product Recommendation System

You are currently modeling the data of a product recommendation system with OrientDB.

The main purpose of such a system is to answer the question "which products were purchased by their people who purchased product X? »

Purchased products have only one name field. They are purchased by people who have a nickname.

When a person buys a product, the date of purchase is stored. 

Instead of working with "anonymous" vertices and arcs, you will use classes. The `create class` command allows you to create custom classes.

The vertex classes must extend V, the arc classes must extend E.

**Q:** Create an `eCommerce` database, and the necessary classes to model the system.

PS : you can view all classes in the database with :

```python
for name in client.command("SELECT name FROM (SELECT expand(classes) FROM metadata:schema)"):
    print(name)
```

In [44]:
db = client.db_create("eCommerce",type=pyorient.DB_TYPE_GRAPH,storage=pyorient.STORAGE_TYPE_MEMORY)

**Q:** Create the following products: `spaghetti`, `bolognese sauce`, `cheese`, `apple`.

In [45]:
client.command("CREATE CLASS Products extends V")

[11]

In [46]:
client.command("create property Products.label String")

[1]

In [47]:
client.command("insert into Products (label) values ('spaghetti')")

In [48]:
client.command("insert into Products (label) values ('bolognese sauce')")

In [49]:
client.command("insert into Products (label) values ('cheese')")

In [50]:
client.command("insert into Products (label) values ('apple')")

**Q:** Create the following people: `peter`, `meredith`.

In [51]:
client.command("CREATE CLASS Customer extends V")

[12]

In [52]:
client.command("create property Products.nickname String")

[2]

In [53]:
client.command("insert into Customer (nickname) values ('peter')")

In [54]:
client.command("insert into Customer (nickname) values ('meredith')")

**Q:** Create the following purchases: 
- peter > spaghetti + cheese on 20/01/2016 
- meredith > cheese + apple + bolognese sauce on 22/01/2016
- peter > spaghetti + bolognese sauce on 27/01/2016


In [55]:
data = [
    ("peter",["spaghetti","cheese"],'20/01/2016'),
    ("meredith",["cheese","apple","bolognese sauce"],'22/01/2016'),
    ("peter",["spaghetti","bolognese sauce"],'27/01/2016')   
]

In [56]:
i=1
for tuple in data:
    for product in tuple[1]:
        query = f"CREATE EDGE edge{i} FROM (SELECT FROM Customer where nickname='{tuple[0]}') TO (SELECT FROM Products where label='{product}') set date='{tuple[2]}'"
        client.command(query)
        i+=1


**Q:** Who bought Bolognese sauce?

In [57]:
[item.nickname for item in client.command("SELECT expand(in()) FROM Products where label='bolognese sauce'")]

['meredith', 'peter']

**Q:** It is possible to link the `out` and `in` navigation functions. What products are purchased with Bolognese sauce? 

In [90]:
set([item.label for item in client.command("SELECT expand(in().out()) FROM Products where label='bolognese sauce'")])

{'apple', 'bolognese sauce', 'cheese', 'spaghetti'}

## Postquisites

Since we create databases in memory, they get destroyed on server shutdown.